## Pytorch có 2 hàm để làm việc với dữ liệu là : torch.utils.data.DataLoader và torch.utils.data.DataSet. Hàm dataset sẽ các ví dụ và nhãn của chúng. DataLoader sẽ áp dụng vòng lặp lên Dataset

In [1]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

d:\anaconda\envs\python3.8\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Ngoài ra, pytorch cung cấp các thư viện có sẵn những bộ dữ liệu có sẵn ứng với từng lĩnh vực trong AI như xử lý ngôn ngữ tự nhiên, thị giác máy tính,... như torchtext, torchvision, torchaudio. Ví dụ sẽ sử dụng bộ dữ liệu IMDB để phân lớp văn bản

In [2]:
!pip install portalocker

## Dưới đây là một ví dụ về xử lý dữ liệu NLP điển hình với tokenizer và vocab. Bước đầu tiên là xây dựng bộ từ điển với tập dữ liệu huấn luyện. Ở đây, chúng tôi sử dụng hàm dựng sẵn build_vocab_from_iterator để chấp nhận trình lặp mang lại danh sách hoặc trình lặp tokens. Người dùng cũng có thể chuyển bất kỳ ký hiệu đặc biệt nào để thêm vào từ vựng.

In [3]:
from torchtext import datasets
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")

train_iter = datasets.IMDB(
    root="data",
    split="train"
)

test_iter = datasets.IMDB(
    root="data",
    split="test"
    
)
train_iter
test_iter

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)


vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"]) #add special tokens

In [20]:
vocab(['here', 'is', 'an', 'example'])

[131, 9, 40, 464]

## Chuẩn bị quy trình xử lý văn bản với tokenzier và vocab. Các text pipelines và label pipelines sẽ được sử dụng để xử lý các chuỗi dữ liệu thô từ các vòng lặp của dataset.

In [21]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

Text pipeline chuyển đổi một chuỗi văn bản thành danh sách các số nguyên dựa trên bảng tra cứu được xác định trong vocab. Label pipeline chuyển đổi nhãn thành số nguyên. 

In [24]:
text_pipeline('here is the an example')
label_pipeline('10')

9

torch.utils.data.DataLoader được khuyến nghị cho người dùng PyTorch. Nó hoạt động với tập dữ liệu kiểu mapping, triển khai các giao thức getitem() và len() và thể hiện mapping từ các chỉ mục/khóa đến mẫu dữ liệu. Nó cũng hoạt động với một tập dữ liệu có thể lặp lại với đối số xáo trộn(shuffle) là False.

Trước khi gửi đến mô hình, hàm collate_fn hoạt động trên một loạt mẫu được tạo từ DataLoader. Đầu vào của collate_fn là một batch dữ liệu có batch_size trong DataLoader và collate_fn xử lý chúng theo quy trình xử lý dữ liệu đã khai báo trước đó. Hãy chú ý ở đây và đảm bảo rằng collate_fn được khai báo là def cấp cao nhất. Điều này đảm bảo rằng chức năng này có sẵn trong mỗi công nhân.

Trong ví dụ này, các văn bản trong dữ liệu đầu vào theo batch ban đầu được đóng gói thành một danh sách và được nối thành một tensor đơn cho đầu vào của nn.EmbeddingBag. Offset là một tensor của các dấu phân cách để biểu thị chỉ số bắt đầu của chuỗi riêng lẻ trong tensor văn bản. Nhãn là một tensor lưu nhãn của các mục văn bản riêng lẻ.

In [25]:
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

dataloader = DataLoader(
    train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch
)

## Pytorch có thể huấn luyện với CPU và GPU

In [4]:
torch.__version__

'1.9.1+cpu'

In [5]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)

cpu


## Xây dựng mô hình
Mô hình này bao gồm lớp nn.EmbeddingBag cộng với lớp tuyến tính cho mục đích phân loại. nn.EmbeddingBag với chế độ mặc định là “trung bình” sẽ tính giá trị trung bình của một “túi” các phần embedding. Mặc dù các mục nhập văn bản ở đây có độ dài khác nhau, mô-đun nn.EmbeddingBag không yêu cầu phần đệm ở đây vì độ dài văn bản được lưu theo độ lệch.

Ngoài ra, vì nn.EmbeddingBag tích lũy nhanh chóng mức trung bình trên các phần nhúng nên nn.EmbeddingBag có thể nâng cao hiệu suất và hiệu quả bộ nhớ để xử lý một chuỗi các tensor.

## 

In [9]:
import torch.nn as nn 

class NeuralNetwork(nn.Module):
    def __init__(self, vocab_size, embedd_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(num_embeddings=vocab_size, embedding_dim=embedd_dim) #Embedding layer
        self.linear = nn.Linear(in_features=embedd_dim, out_features=num_class) #Linear Layer
        self.init_weight()
    
    def init_weight(self): # initialize weight metris
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.linear.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        
    def forward(self, x):
        x = self.embedding(x)
        output = self.linear(x)
        
        return output

In [10]:
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)

embedding_size = 64
model = NeuralNetwork(vocab_size=vocab_size, embedd_dim=embedding_size, num_class=num_class)

model.to(device)

model

d:\anaconda\envs\python3.8\lib\site-packages\torch\nn\init.py:388: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


NeuralNetwork(
  (embedding): EmbeddingBag(100683, 64, mode=mean)
  (linear): Linear(in_features=64, out_features=0, bias=True)
)

## Thuật toán tối ưu và hàm mất mát

In [11]:
import torch.optim as optim

lr = 1e-3
optimizer = optim.Adam(model.parameters(),lr=lr)

loss_function = nn.CrossEntropyLoss()

## Bước huấn luyện mô hình

In [12]:
def train(dataloader):
    model.train() #mode of model
    total_acc = 0
    total_count = 0
    
    log_interval = 500
    
    for batch, (label, text, offsets) in enumerate(dataloader):
        #model prediction
        pred = model(text, offsets)
        
        #compute loss
        loss = loss_function(pred, label)
        
        #backprobagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        total_acc += (pred.argmax(1)==label).sum().item()
        total_count += label.size(0)
        if batch % log_interval == 0 and batch > 0:
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, batch, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0

## Bước test mô hình

In [13]:
def test(dataloader):
    model.eval() #mode of model
    
    total_acc = 0
    total_count = 0
    with torch.no_grad():
        for batch, (label, text, offsets) in enumerate(dataloader):        
                pred = model(text)
                
                loss += loss_function(pred, label)
                total_acc += (pred.argmax(1) == label).sum().item()
                total_count += label.size(0)
    return total_acc/total_count

Quá trình huấn luyện được thực hiện qua nhiều lần lặp (epoch). Trong mỗi epoch, mô hình sẽ điều chỉnh các tham số để đưa ra dự đoán tốt hơn. Chúng tôi in độ chính xác và độ mất mát của mô hình tại mỗi epoch; chúng tôi muốn thấy độ chính xác tăng lên và độ mất mát giảm dần theo từng epoch.

In [16]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 2  # epoch
LR = 5  # learning rate
BATCH_SIZE = 64  # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = datasets.IMDB()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.20)
split_train_, split_valid_ = random_split(
    train_dataset, [num_train, len(train_dataset) - num_train]
)

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)

for epoch in range(1, EPOCHS + 1):
    train(train_dataloader)
    accu_val = test(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | "
        "valid accuracy {:8.3f} ".format(
            epoch, accu_val
        )
    )
    print("-" * 59)

RuntimeError: The hash of d:\Learn_at_free_time\.data\IMDB\aclImdb_v1.tar.gz does not match. Delete the file manually and retry.